In [33]:
from scipy import stats
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from sklearn.ensemble import IsolationForest

In [34]:
df = pd.read_csv(r'data\AdSmartABdata - AdSmartABdata.csv')

# Examinando os dados


In [35]:
# Exibindo as primeiras linhas dos dados
df.head()

,auction_id,experiment,date,hour,device_make,platform_os,browser,yes,no
0,0008ef63-77a7-448b-bd1e-075f42c55e39,exposed,2020-07-10,8,Generic Smartphone,6,Chrome Mobile,0,0
1,000eabc5-17ce-4137-8efe-44734d914446,exposed,2020-07-07,10,Generic Smartphone,6,Chrome Mobile,0,0
2,0016d14a-ae18-4a02-a204-6ba53b52f2ed,exposed,2020-07-05,2,E5823,6,Chrome Mobile WebView,0,1
3,00187412-2932-4542-a8ef-3633901c98d9,control,2020-07-03,15,Samsung SM-A705FN,6,Facebook,0,0
4,001a7785-d3fe-4e11-a344-c8735acacc2c,control,2020-07-03,15,Generic Smartphone,6,Chrome Mobile,0,0


In [36]:
# Verificando os tipos de dados das colunas
df.dtypes

auction_id     object
experiment     object
date           object
hour            int64
device_make    object
platform_os     int64
browser        object
yes             int64
no              int64
dtype: object

In [37]:

# Verificando se há valores ausentes
df.isnull().sum()

auction_id     0
experiment     0
date           0
hour           0
device_make    0
platform_os    0
browser        0
yes            0
no             0
dtype: int64

In [38]:
# Verificando os valores únicos para cada variável categórica
print(df['experiment'].unique())
print('\n')
print(df['device_make'].unique())
print('\n')
print(df['platform_os'].unique())
print('\n')
print(df['browser'].unique())

['exposed' 'control']


['Generic Smartphone' 'E5823' 'Samsung SM-A705FN' 'Samsung SM-G960F'
 'Samsung SM-G973F' 'iPhone' 'Samsung SM-G935F' 'HTC One' 'LG-$2'
 'Samsung SM-A202F' 'XT1032' 'COL-L29' 'Samsung SM-N960U1'
 'Samsung SM-A715F' 'Samsung SM-G930F' 'I3312' 'Samsung SM-G950F'
 'FIG-LX1' 'Samsung SM-G920F' 'MRD-LX1' 'Samsung SM-N950F' 'Moto $2'
 'Samsung SM-G970F' 'Samsung GT-I9505' 'Samsung SM-G981B' 'Pixel 3a'
 'Samsung SM-J600FN' 'Samsung SM-A105FN' 'OnePlus ONEPLUS A3003' 'POT-LX1'
 'Samsung SM-G975F' 'Samsung SM-J330FN' 'Samsung SM-G770F' 'H3311'
 'MAR-LX1A' 'HTC One $2' 'Samsung SM-G965F' 'ELE-L09' 'Samsung SM-J415FN'
 'Samsung SM-G900F' 'Lenovo A1010a20' 'CLT-L09' 'HTC Desire $2'
 'Samsung SM-G980F' 'Samsung SM-G955F' 'Samsung SM-N960F' 'Nexus 5'
 'Samsung SM-J260F' 'HTC U11' 'Samsung SM-A405FN' 'Samsung SM-A600FN'
 'ANE-LX1' 'VOG-L09' 'Samsung SM-G986B' 'XiaoMi Redmi Note 4' 'ELE-L29'
 'Samsung $2' 'Samsung SM-A320FL' 'OnePlus ONEPLUS A5000'
 'Samsung SM-A505FN' 'Nokia un

In [39]:
# Convertendo as colunas de data e hora para um objeto de data e hora
df['date'] = pd.to_datetime(df['date'])
df['hour'] = pd.to_datetime(df['hour'], format='%H').dt.time

# Criando uma nova coluna que combina as colunas de data e hora
df['date_hour'] = pd.to_datetime(df['date'].astype(str) + ' ' + df['hour'].astype(str))

# Removendo as colunas originais de data e hora
df = df.drop(['date', 'hour'], axis=1)

# Substituindo os valores ausentes por zeros
df = df.fillna(0)

In [40]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8077 entries, 0 to 8076
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   auction_id   8077 non-null   object        
 1   experiment   8077 non-null   object        
 2   device_make  8077 non-null   object        
 3   platform_os  8077 non-null   int64         
 4   browser      8077 non-null   object        
 5   yes          8077 non-null   int64         
 6   no           8077 non-null   int64         
 7   date_hour    8077 non-null   datetime64[ns]
dtypes: datetime64[ns](1), int64(3), object(4)
memory usage: 504.9+ KB


# Criando um histograma a partir dos dados contidos no DataFrame

O código abaixo cria um histograma utilizando a biblioteca Plotly, com base nos dados do DataFrame 'df'. O histograma é plotado usando a coluna 'yes' como eixo x e a coluna 'experiment' como cor das barras.

Os parâmetros utilizados são os seguintes:

- 'nbins': Define o número de bins (intervalos) no histograma.
- 'category_orders': É um dicionário que especifica a ordem das categorias na coluna 'yes'.
- 'labels': É um dicionário que define os rótulos das categorias.
- 'title': Define o título do gráfico como 'Distribuição das Respostas'.
- 'color_discrete_sequence': Define a sequência de cores para as barras do histograma.
Após a criação do histograma, são feitas algumas atualizações no layout do gráfico, como a adição de um título para a legenda ('Grupo'), a orientação da legenda ('orientation'), a posição da legenda ('yanchor', 'y', 'xanchor', 'x'), e a adição de anotações com os valores de contagem em cada barra do histograma.

Por fim, o gráfico é exibido utilizando o método 'fig.show()'.

In [41]:

fig = px.histogram(df, x='yes', color='experiment', nbins=2,
                   category_orders={'yes': ['No', 'Yes']},
                   labels={'yes': 'Resposta', 'count': 'Contagem'},
                   title='Distribuição das Respostas',
                   color_discrete_sequence=['#1f77b4', '#ff7f0e'])
fig.update_layout(legend_title='Grupo', legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='right', x=1))
for trace in fig.data:
    if 'count' not in trace:
        continue
    for i, count in enumerate(trace.y):
        fig.add_annotation(x=trace.x[i], y=count, text=str(count), showarrow=False, font=dict(size=12))

fig.show()

In [42]:
# Agrupa os dados por experimento e calcula a taxa média de cliques para cada grupo
ctr_por_grupo = df.groupby('experiment')['yes'].mean().reset_index()

# Cria um traço de gráfico de barras
bar = go.Bar(x=ctr_por_grupo['experiment'], y=ctr_por_grupo['yes'],
             text=ctr_por_grupo['yes'].apply(lambda x: f"{x:.2%}"), textposition='auto',
             marker_color=['#1f77b4', '#ff7f0e'], opacity=0.8)

# Cria um objeto de layout
layout = go.Layout(title='Taxa de cliques por grupo', xaxis_title='Group',
                   yaxis_title='Taxa de cliques', yaxis_range=[0, 0.2],
                   xaxis=dict(tickmode='array', tickvals=[0, 1], ticktext=['Controlado', 'Exposto']))

# Cria um objeto de figura que inclui o traço do gráfico de barras e o layout
fig = go.Figure(data=[bar], layout=layout)

# Mostra o gráfico
fig.show()

O código abaixo cria um gráfico de dispersão (scatter plot) com dados de um DataFrame chamado df. O gráfico possui os seguintes elementos:

- Eixo x (x='date_hour'): Os valores do eixo x são obtidos da coluna "date_hour" do DataFrame df, que representa a data e hora.
- Eixo y (y='yes'): Os valores do eixo y são obtidos da coluna "yes" do DataFrame df, que representa uma taxa de cliques (Taxa de cliques).
- Cor (color='experiment'): Os pontos no gráfico são coloridos com base nos valores da coluna "experiment" do DataFrame df, que representa diferentes grupos ou experimentos.
- Opacidade (opacity=0.5): A opacidade dos pontos no gráfico é definida como 0,5, o que significa que eles são parcialmente transparentes.
- Rótulos dos eixos (labels={'date_hour': 'Data e Hora', 'yes': 'Taxa de cliques'}): Os rótulos dos eixos x e y são personalizados para "Data e Hora" e "Taxa de cliques", respectivamente.
- Título do gráfico (title='Taxas de cliques ao longo do tempo'): O título do gráfico é definido como "Taxas de cliques ao longo do tempo".
- Configuração do layout (fig.update_layout(...)): Algumas configurações adicionais são aplicadas ao layout do gráfico, incluindo a rotação dos rótulos do eixo x em 45 graus, o ajuste da faixa do eixo y para variar de -0,1 a 1,1, a definição do título da legenda como "Group" e a personalização da posição e orientação da legenda.
Por fim, a função fig.show() é chamada para exibir o gráfico criado usando a biblioteca Plotly Express.

In [43]:
fig = px.scatter(df, x='date_hour', y='yes', color='experiment', opacity=0.5,
                 labels={'date_hour': 'Data e Hora', 'yes': 'Taxa de cliques'},
                 title='Taxas de cliques ao longo do tempo')

fig.update_layout(xaxis=dict(tickangle=45, tickfont=dict(size=8), title_font=dict(size=12)),
                  yaxis_range=[-0.1, 1.1], legend_title='Grupo',
                  legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='right', x=1))

fig.show()

In [44]:
# Calcula o número de usuários em cada grupo
tamanho_controle = len(df[df['experiment'] == 'control'])
tamanho_exposto = len(df[df['experiment'] == 'exposed'])

# Calcula as taxas de cliques para cada fabricante de dispositivo por grupo
taxa_cliques_por_fabricante = df.groupby(['device_make', 'experiment'])[['yes']].mean().reset_index()

# Cria um gráfico de mapa de calor
mapa_calor = go.Heatmap(x=taxa_cliques_por_fabricante['experiment'], y=taxa_cliques_por_fabricante['device_make'], z=taxa_cliques_por_fabricante['yes'],
colorscale='Blues', zmin=0, zmax=1, showscale=True, reversescale=True)

# Cria um layout
layout = go.Layout(title='Taxas de cliques por fabricante de dispositivo e grupo',
xaxis=dict(title='Grupo'),
yaxis=dict(title='Fabricante de dispositivo', autorange='reversed'),
margin=dict(l=150, r=50, b=200, t=100))

# Cria uma figura e a exibe
fig = go.Figure(data=[mapa_calor], layout=layout)
fig.show()

In [45]:
# Cria um novo dataframe apenas com a coluna 'yes'
yes_df = df[['yes']]

# Cria uma instância do algoritmo Isolation Forest
iforest = IsolationForest(n_estimators=100, contamination=0.01, random_state=42)

# Ajusta o algoritmo aos dados
iforest.fit(yes_df)

# Adiciona uma nova coluna ao dataframe original com os escores de anomalia
df['outlier_score'] = iforest.decision_function(yes_df)

# Imprimie o número de outliers detectados
num_outliers = len(df[df['outlier_score'] < 0])
print(f"Número de outliers detectados: {num_outliers}")

c:\Users\lucas\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:439: UserWarning:

X does not have valid feature names, but IsolationForest was fitted with feature names



Número de outliers detectados: 0


## Análise Estatística

O código abaixo realiza uma análise de teste t para comparar as taxas de sucesso (proporções) de duas amostras, uma de controle e outra exposta a um experimento, para diferentes categorias de um atributo chamado 'device_make' em um dataframe.

O código começa criando uma lista única de valores únicos da variável 'device_make' no dataframe 'df'. Em seguida, um loop é executado para cada valor único de 'device_make'. Dentro do loop, as taxas de sucesso (proporções) para as amostras de controle e exposição são calculadas usando a função 'mean()' do pandas para calcular a média dos valores 'yes' (ou seja, o sucesso) para cada grupo de controle e exposição.

Em seguida, o tamanho das amostras de controle e exposição é calculado usando a função 'len()' do pandas para contar o número de observações em cada grupo. Com base nos tamanhos das amostras, as estatísticas do teste t, como erro padrão (se), diferença entre as taxas de sucesso (diff), estatística t (t_stat) e valor de p (p_value), são calculadas usando as fórmulas estatísticas apropriadas.

Os resultados são armazenados em uma lista chamada 'results' e, em seguida, convertidos em um dataframe chamado 'results_df' usando a função 'pd.DataFrame()'. Os resultados são ordenados em ordem crescente com base nos valores de p usando a função 'sort_values()' do pandas. Os resultados são exibidos usando a função 'print()'.

In [46]:
dispositivos = df['device_make'].unique()
results = []
for marca in dispositivos:
    control_ctr = df[(df['device_make'] == marca) & (df['experiment'] == 'control')]['yes'].mean()
    exp_ctr = df[(df['device_make'] == marca) & (df['experiment'] == 'exposed')]['yes'].mean()
    control_tam = len(df[(df['device_make'] == marca) & (df['experiment'] == 'control')])
    exp_tam = len(df[(df['device_make'] == marca) & (df['experiment'] == 'exposed')])
    if control_tam > 0 and exp_tam > 0:
        se = np.sqrt((control_ctr * (1 - control_ctr) / control_tam) + (exp_ctr * (1 - exp_ctr) / exp_tam))
        diff = exp_ctr - control_ctr
        t_stat = diff / se
        p_value = stats.t.sf(np.abs(t_stat), min(control_tam, exp_tam)-1) * 2
    else:
        se, diff, t_stat, p_value = np.nan, np.nan, np.nan, np.nan
    results.append([marca, control_ctr, exp_ctr, diff, se, t_stat, p_value])

# Cria um dataframe com os resultados do teste t
results_df = pd.DataFrame(results, columns=['device_make', 'control_ctr', 'exp_ctr', 'diff', 'se', 't_stat', 'p_value'])

# Ordena os resultados pelo valor de p em ordem crescente
results_df = results_df.sort_values('p_value')

# Exibe os resultados
print(results_df)

C:\Users\lucas\AppData\Local\Temp\ipykernel_14116\1965183851.py:11: RuntimeWarning:

invalid value encountered in scalar divide

C:\Users\lucas\AppData\Local\Temp\ipykernel_14116\1965183851.py:11: RuntimeWarning:

divide by zero encountered in scalar divide



             device_make  control_ctr   exp_ctr      diff        se    t_stat  \
0     Generic Smartphone     0.057652  0.078045  0.020392  0.007307  2.790869   
14      Samsung SM-G930F     0.175439  0.023256 -0.152183  0.055373 -2.748330   
36      Samsung SM-G965F     0.034483  0.189189  0.154706  0.072759  2.126277   
75      Samsung SM-J530F     0.066667  0.312500  0.245833  0.132574  1.854309   
18      Samsung SM-G920F     0.000000  0.100000  0.100000  0.054772  1.825742   
..                   ...          ...       ...       ...       ...       ...   
264     Samsung SM-G9730          NaN  0.000000       NaN       NaN       NaN   
265    Samsung SM-G955U1          NaN  0.000000       NaN       NaN       NaN   
266      XiaoMi Redmi S2     1.000000       NaN       NaN       NaN       NaN   
267     Samsung SM-A205F     0.000000       NaN       NaN       NaN       NaN   
268  XiaoMi Redmi Note 5          NaN  0.000000       NaN       NaN       NaN   

      p_value  
0    0.0052

**Conclusão**: A hipótese nula é que não há diferença nas taxas de cliques entre os grupos de controle e exposição para cada fabricante de dispositivo, e a hipótese alternativa é que há diferença nas taxas de cliques entre os grupos de controle e exposição para pelo menos um fabricante de dispositivo.

Também podemos especificar o nível de significância para os testes t, que é a probabilidade de rejeitar a hipótese nula quando ela é verdadeira. Um nível de significância comum é 0,05, o que significa que temos uma chance de 5% de cometer um erro do Tipo I (rejeitar a hipótese nula quando ela é verdadeira).

O código abaixo cria um gráfico de box plot usando a biblioteca Plotly, com o eixo x representando o navegador utilizado, o eixo y representando a taxa de cliques (variável 'yes'), e a cor representando o grupo de experimento ('control' ou 'exposed'). Os rótulos do eixo x e y são atualizados para "Navegador" e "Taxa de cliques", respectivamente. O título do gráfico é definido como "Taxa de cliques por grupo e por navegador". O layout do gráfico é atualizado para ajustar o ângulo dos rótulos do eixo x, definir a faixa do eixo y entre -0,1 e 1,1, e personalizar a legenda do gráfico.

In [47]:
# Create a box plot trace
box = px.box(df, x='browser', y='yes', color='experiment',
             labels={'browser': 'Navegador', 'yes': 'Taxa de cliques'},
             title='Taxa de cliques por grupo e por navegador')

# Update the layout
box.update_layout(xaxis_tickangle=90, yaxis_range=[-0.1, 1.1],
                  legend_title='Grupo', legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='right', x=1))

# Show the plot
box.show()

In [48]:
# Calcula as taxas de cliques gerais para os grupos expostos e de controle
exp_ctr = df[df['experiment']=='exposed']['yes'].mean()
con_ctr = df[df['experiment']=='control']['yes'].mean()

print("Taxa de cliques")
print("Grupo Exposto: ", exp_ctr)
print("Grupo Controlado: ", con_ctr)
print("Diferença: ", exp_ctr - con_ctr)

# Calcula as taxas de cliques por marca de dispositivo
ctr_por_dispositivo = df.groupby(['device_make', 'experiment'])[['yes']].mean().reset_index()

# Imprime as marcas de dispositivo com as maiores taxas de cliques para o grupo exposto
top_dispositivos = ctr_por_dispositivo[ctr_por_dispositivo['experiment']=='exposed'].sort_values(by='yes', ascending=False).head()
print("\nPrincipais marcas de dispositivos com a maior taxa de cliques para o grupo exposto:")
print(top_dispositivos)

# Calcula as taxas de cliques por navegador
ctr_por_dispositivo = df.groupby(['browser', 'experiment'])[['yes']].mean().reset_index()

# Imprime os navegadores com as maiores taxas de cliques para o grupo exposto
top_navegadores = ctr_por_dispositivo[ctr_por_dispositivo['experiment']=='exposed'].sort_values(by='yes', ascending=False).head()
print("\nPrincipais navegadores com maior taxa de cliques para o grupo exposto:")
print(top_navegadores)

# Calcula as taxas de cliques por data e grupo
ctr_por_data = df.groupby(['date_hour', 'experiment'])[['yes']].mean().reset_index()

# Cria traços de gráfico de linha para cada grupo
linha_exp = px.line(ctr_por_data[ctr_por_data['experiment']=='exposed'], x='date_hour', y='yes',
                   labels={'date_hour': 'Data e Hora', 'yes': 'Taxa de cliques'},
                   title='Taxas de cliques ao longo do tempo para o grupo exposto')
linha_control = px.line(ctr_por_data[ctr_por_data['experiment']=='control'], x='date_hour', y='yes',
                   labels={'date_hour': 'Data e Hora', 'yes': 'Taxa de cliques'},
                   title='Taxas de cliques ao longo do tempo para o grupo de controle')

# Atualiza o layout
linha_exp.update_layout(xaxis_tickformat='%Y-%m', yaxis_range=[0, 0.2], legend_title='Group',
                       legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='right', x=1))
linha_control.update_layout(xaxis_tickformat='%Y-%m', yaxis_range=[0, 0.2], legend_title='Group',
                       legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='right', x=1))

# Mostra os gráficos
linha_exp.show()
linha_control.show()

Taxa de cliques
Grupo Exposto:  0.07688467299051423
Grupo Controlado:  0.06484893146647015
Diferença:  0.012035741524044075

Principais marcas de dispositivos com a maior taxa de cliques para o grupo exposto:
               device_make experiment  yes
337                VFD 700    exposed  1.0
4                    A0001    exposed  1.0
332                VCE-L22    exposed  1.0
69           HTC Desire $2    exposed  1.0
113  OnePlus ONEPLUS A3000    exposed  1.0

Principais navegadores com maior taxa de cliques para o grupo exposto:
                  browser experiment       yes
2                  Chrome    exposed  0.500000
6   Chrome Mobile WebView    exposed  0.081871
4           Chrome Mobile    exposed  0.079291
11               Facebook    exposed  0.078818
23       Samsung Internet    exposed  0.066265


# Conclusão

Com base na análise, o anúncio interativo online com a **marca SmartAd** foi considerado *mais eficaz do que* o **anúncio fictício** na geração de *engajamento do usuário e cliques*. **A taxa de cliques para o grupo exposto foi de 0,0769, que é maior** do que a taxa de cliques para o grupo de **controle de 0,0648**. *A diferença entre os dois grupos é de 0,0120*, indicando que o *anúncio interativo online foi mais eficaz na geração de engajamento do usuário.*

As *principais marcas de dispositivos* com as maiores taxas de cliques para o grupo exposto foram **VFD 700, A0001, VCE-L22, HTC Desire 2, e OnePlus ONEPLUS A3000**. Os *principais navegadores com as maiores taxas de cliques para o grupo exposto **foram Chrome, Chrome Mobile WebView, Chrome Mobile, Facebook, e Samsung Internet**. 

Esses resultados sugerem que o anúncio interativo online com **a marca SmartAd pode ser mais eficaz para determinados tipos de dispositivos e navegadores, e que direcionar esses tipos específicos de dispositivos e navegadores pode resultar em taxas de engajamento mais altas.**